# PROGRAMA DE CIENCIAS DE LOS DATOS 
# **Curso: Big Data**
## **PROYECTO FINAL:**

## <font color='red'>Machine Learning con datos en PostgreSQL</font>. 


#### **Profesor: MSc. Felipe Meza Obando**


#### Alumnos: 
  
####  **Lester Salazar Viales.**
####  **Randal Salazar Viales.**


### Objetivo del Proyecto

Se efectuará un Análisis de datos de Machine Learning, a una base de datos que se encuentra ubicada en una **BD PostgreSQL**.

Para la BD a emplear, se efectuará un **análisis predictorio**, para tratar de **determinar la cantidad de días de vacaciones que  posee cada empleado**, de acuerdo a las features existentes en la BD.

## <font color='red'> DATASET DE RH DE UN BANCO </font>


#### El conjunto de datos brinda información sobre el personal que labora para un banco.

#### **Objetivo**: Tratar de determinar si es posible obtener la cantidad de días de vacaciones a su disposición, para cada uno de los empleado del Banco .

### A. DESCRIPCIÓN DE ATRIBUTOS:

**Variables de Entrada:**

#### A. Datos de empleados del banco:

1 - **cedula**: número de cédula del empleado (numérica)
    
2 - **cod_planilla**: código de planilla a la que pertenece el empleado en el banco (categórica)

3 - **fecha_ingreso**: fecha de ingreso a laborar en el banco por el empleado (categórica)

4 - **fecha_nacimiento**: fecha de nacimiento del empleado contratado (categórica)

5 - **jornada_trabajo**: jornada de trabajo del empleado (categórica: A=DIURNA L-V, B=MIXTA L-V, C=NOCHE  L-V).

6 - **cod_puesto**: código del puesto de trabajo del empleado (categórica: 849 valores posibles)

7 - **sexo**: sexo del empleado (categórica binaria: 'M','F')
    
8 - **cod_concepto_salarial**: código del concepto salarial del ingreso/egreso de dinero del empleado (categórica)
    
9 - **cod_area**: tiene crédito en incumplimiento? (categórica: 'no','yes','unknown')
    
10 - **dias_derecho**: días que puede disfrutar de vacaciones el empleado hasta este momento (numérica).
    
11 - **monto_aplicar**: dinero devengado por el empleado (numérica)
    
12 - **provincia**: provincia donde labora el empleado (numérica: 1 , 2, 3, 4, 5, 6, 7)
    
14 - **dias_pendiente**: días que puede disfrutar de vacaciones el empleado hasta este momento (numérica). 
        valor positivo: son los días que le quedan por disfrutar al empleado.
        valor negativo: son los días que a disfrutado el empleado demás con respecto al período.
    
15 - **edad**: edad del empleado (numérica).

16 - **estado_civil**: estado civil del empleado (categórica: C=Casado, D=Divorciado, S=Soltero, U=Unión Libre, V=Viudo).



**Variable de Salida (objetivo deseado):**

21 - **vacaciones** - la cantidad de días de vacaciones que dispone el empleado (numérica)

## <font color='red'>**Regresión Lineal:</font> <font color='blue'>vacaciones**</font>

El dataset contiene $m=5109 $ muestras u observaciones.

### Importación de Librerías

In [1]:
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import six

import seaborn as sns
sns.set(color_codes=True)
import numpy as np
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, SQLTransformer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col,sum

### Creación de SparkSession

In [2]:
import findspark
findspark.init('C:\spark')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf, when, lit
from pyspark.sql.types import DateType

# Importación de Librerías para Machine Learning
from pyspark.sql import functions as F
from pyspark.sql import *

spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "C:\Spark\jdbcdriver\postgresql-42.2.9.jar") \
    .config("spark.executor.extraClassPath", "C:\Spark\jdbcdriver\postgresql-42.2.9.jar") \
    .getOrCreate()

### Conexión a BD PostgreSQL mediante Spark

### Creación del DataFrame de datos de BD empleada

- **Dataframe de Tabla Empleados**

- Cargado de datos y visualización parcial del dataset

In [3]:
# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost/rhdb") \
    .option("user", "postgres") \
    .option("password", "bd2019%") \
    .option("dbtable", "empleado_final") \
    .load()

df.toPandas().head(4).transpose()

,0,1,2,3
cedula,0108150330,0110440601,0114460350,0111840955
cod_planilla,Q,Q,Q,Q
fecha_ingreso,28/10/1994,2/4/2004,25/3/2011,2/2/2004
fecha_nacimiento,1972-01-26,1979-08-14,1990-09-29,1983-10-12
jornada_trabajo,A,A,A,A
cod_puesto,GZO1.13,PRC4.11,ASB3.16,PRC4.11
sexo,M,M,M,F
cod_concepto_salarial,41,41,41,41
cod_area,51,51,51,51
dias_derecho,30.000000000000000000,30.000000000000000000,18.000000000000000000,30.000000000000000000


#### Total de datos del dataframe: filas, columnas

In [4]:
#check the shape of the data
print('Cantidad de: datos, atributos:')
print((df.count(),len(df.columns)))

Cantidad de: datos, atributos:
(5109, 15)


- **Visualización del esquema del DataFrame**:

In [5]:
df.printSchema()

root
 |-- cedula: string (nullable = true)
 |-- cod_planilla: string (nullable = true)
 |-- fecha_ingreso: string (nullable = true)
 |-- fecha_nacimiento: string (nullable = true)
 |-- jornada_trabajo: string (nullable = true)
 |-- cod_puesto: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cod_concepto_salarial: string (nullable = true)
 |-- cod_area: string (nullable = true)
 |-- dias_derecho: decimal(38,18) (nullable = true)
 |-- monto_aplicar: decimal(38,18) (nullable = true)
 |-- provincia: integer (nullable = true)
 |-- dias_pendiente: decimal(38,18) (nullable = true)
 |-- edad: integer (nullable = true)
 |-- estado_civil: string (nullable = true)



- **Nombres de columnas del DataFrame**:

In [6]:
df.columns

['cedula',
 'cod_planilla',
 'fecha_ingreso',
 'fecha_nacimiento',
 'jornada_trabajo',
 'cod_puesto',
 'sexo',
 'cod_concepto_salarial',
 'cod_area',
 'dias_derecho',
 'monto_aplicar',
 'provincia',
 'dias_pendiente',
 'edad',
 'estado_civil']

- **Cambio de nombre de columnas: monto_aplicar por salario**

In [7]:
df = df.withColumnRenamed('monto_aplicar', 'salario')

- Visualización del esquema del DataFrame por el cambio en nombre columna:

In [8]:
df.printSchema()

root
 |-- cedula: string (nullable = true)
 |-- cod_planilla: string (nullable = true)
 |-- fecha_ingreso: string (nullable = true)
 |-- fecha_nacimiento: string (nullable = true)
 |-- jornada_trabajo: string (nullable = true)
 |-- cod_puesto: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cod_concepto_salarial: string (nullable = true)
 |-- cod_area: string (nullable = true)
 |-- dias_derecho: decimal(38,18) (nullable = true)
 |-- salario: decimal(38,18) (nullable = true)
 |-- provincia: integer (nullable = true)
 |-- dias_pendiente: decimal(38,18) (nullable = true)
 |-- edad: integer (nullable = true)
 |-- estado_civil: string (nullable = true)



## Análisis Exploratorio de Datos (EDA)

### -  Detección de valores nulos (missing values) dentro del dataframe

In [9]:
df_null = df.select(*(F.sum(F.col(c).isNull().cast('double')).alias(c) for c in df.columns)).toPandas()
df_null.transpose()

,0
cedula,0.0
cod_planilla,0.0
fecha_ingreso,0.0
fecha_nacimiento,0.0
jornada_trabajo,0.0
cod_puesto,0.0
sexo,0.0
cod_concepto_salarial,0.0
cod_area,0.0
dias_derecho,0.0


Como se puede apreciar, el dataframe NO contiene valores nulos en ninguna de sus columnas.

### - Detección de valores únicos dentro del dataframe

In [10]:
# Eliminación del ID del dataframe:
#df_select1 = df.columns[1:]
df_select1 = df.columns
print(' ')
print('Nombres de columnas del DataFrame:  ')
print(df_select1)
print(' ')

# Detección de valores únicos
print('Cantidad de datos únicos por columnas del DataFrame:  ')
for col in df_select1:
    col_count = df.select(col).distinct().count()
    print('variable: {0} , count {1}'.format(col, col_count))

 
Nombres de columnas del DataFrame:  
['cedula', 'cod_planilla', 'fecha_ingreso', 'fecha_nacimiento', 'jornada_trabajo', 'cod_puesto', 'sexo', 'cod_concepto_salarial', 'cod_area', 'dias_derecho', 'salario', 'provincia', 'dias_pendiente', 'edad', 'estado_civil']
 
Cantidad de datos únicos por columnas del DataFrame:  
variable: cedula , count 4037
variable: cod_planilla , count 1
variable: fecha_ingreso , count 1905
variable: fecha_nacimiento , count 3442
variable: jornada_trabajo , count 1
variable: cod_puesto , count 578
variable: sexo , count 3
variable: cod_concepto_salarial , count 9
variable: cod_area , count 15
variable: dias_derecho , count 7
variable: salario , count 344
variable: provincia , count 7
variable: dias_pendiente , count 172
variable: edad , count 47
variable: estado_civil , count 5


De este análisis preliminar se aprecia lo siguiente:

- Las columnas **cod_planilla** y **jornada_trabajo** poseen un valor único, por lo tanto **NO aportan nada al modelado**.

- La fecha de nacimiento viene incluida implícitamente en la edad (no se tomará en cuenta para modelado).

- Las columnas **dias_derecho** y **dias_pendiente** no se emplearán en el modelado. La columna dias_pendiente permite calcular la cantidad de **dias_vacaciones** de cada empleado, que será la variable a predecir (label).

- Existen datos erróneos en el features **sexo**, el cual debería ser 2: M - F:

### Corrección de datos erróneos en columna sexo:

De acuerdo a la verificación preliminar anterior, se aprecia que existe 3 datos para la columna SEXO, lo cual es erróneo.

In [11]:
df.groupBy('sexo').count().show()

+----+-----+
|sexo|count|
+----+-----+
|   F| 2330|
|  M7|    1|
|   M| 2778|
+----+-----+



Existe un dato erróneo: M7. El dato se cambiará por M.

In [12]:
df = df.withColumn('sexo', when(F.col('sexo') == 'M7', 'M').otherwise(F.col('sexo')))

In [13]:
df.groupBy('sexo').count().show()

+----+-----+
|sexo|count|
+----+-----+
|   F| 2330|
|   M| 2779|
+----+-----+



### Cálculo de cantidad de días de vacaciones disponibles de cada empleado

Este feature se calcula a partir de la columna dias_pendientes

- Cantidad de dias_pendientes < 0

Si dias_pendiente < 0, significa que el empleado ha tomado esos días demás con respecto a los dias_derecho que disponía hasta ese período

In [14]:
df.filter(df['dias_pendiente'] < 0).count()

588

El cálculo de días de vacaciones viene en función de la columna **dias_pendiente**.

dias_derecho = días que puede disfrutar de vacaciones.

dias_pendiente = días que dispone para vacaciones. Si el valor el negativo, es porque ha gastado más de las que dispone en ese momento como acreditadas.



Se calcula de la siguiente forma:

    Si dias_pendiente < 0, Vacaciones = 0, de lo contrario
                            Vacaciones = dias_pendiente

Por tanto, dias_pendiente < 0, la persona a tomado más días de vacaciones de las que tiene disponible.

In [15]:
df = df.withColumn(
    'dias_vacaciones', when(F.col('dias_pendiente') < 0, lit(0).cast('float'))
    .otherwise(F.col('dias_pendiente')).cast('float'))

Visualización de cómo queda el Schema (tipo de dato de la columna dias_vacaciones creada)

In [16]:
df.printSchema()

root
 |-- cedula: string (nullable = true)
 |-- cod_planilla: string (nullable = true)
 |-- fecha_ingreso: string (nullable = true)
 |-- fecha_nacimiento: string (nullable = true)
 |-- jornada_trabajo: string (nullable = true)
 |-- cod_puesto: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cod_concepto_salarial: string (nullable = true)
 |-- cod_area: string (nullable = true)
 |-- dias_derecho: decimal(38,18) (nullable = true)
 |-- salario: decimal(38,18) (nullable = true)
 |-- provincia: integer (nullable = true)
 |-- dias_pendiente: decimal(38,18) (nullable = true)
 |-- edad: integer (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- dias_vacaciones: float (nullable = true)



- Visualización parcial de los datos para las columnas dias:

In [17]:
df.select(['dias_derecho', 'dias_pendiente', 'dias_vacaciones']).show(5)

+--------------------+--------------------+---------------+
|        dias_derecho|      dias_pendiente|dias_vacaciones|
+--------------------+--------------------+---------------+
|30.00000000000000...|31.00000000000000...|           31.0|
|30.00000000000000...|13.00000000000000...|           13.0|
|18.00000000000000...|5.000000000000000000|            5.0|
|30.00000000000000...|-3.50000000000000...|            0.0|
|30.00000000000000...|36.00000000000000...|           36.0|
+--------------------+--------------------+---------------+
only showing top 5 rows



- Verificación de que NO existen valores negativos en columna **dias_vacaciones**

a) Cantidad de dias_vacaciones = 0

In [18]:
df.filter(df['dias_vacaciones'] == 0).count()

704

- Verificación de si existe días de vacaciones negativos:

In [19]:
df.filter(df['dias_vacaciones'] < 0).count()

0

b) Cantidad de dias_pendiente < 0

In [20]:
df.filter(df['dias_pendiente'] < 0).count()

588

c) Cantidad de dias en los que se debe tener vacaciones 0 0 (se ha gastado el total de días que puede disfrutar)

In [21]:
df.filter(df['dias_pendiente'] == 0).count()

116

Como b) + c) = a), la conversión de datos está bien.

### - Descripción de los datos

In [22]:
# Descripción de features del dataframe:
df_select1 = df.columns

df_describe = df.select([c for c in df_select1]).describe().toPandas().transpose()
df.select([c for c in df_select1]).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
cedula,5109,2.3404361045292622E8,1.943004107621378E8,0017081120,0901350103
cod_planilla,5109,None,None,Q,Q
fecha_ingreso,5109,None,None,1/1/1988,9/9/2019
fecha_nacimiento,5109,None,None,1953-09-14,2001-01-13
jornada_trabajo,5109,None,None,A,A
cod_puesto,5109,None,None,ASB1.01,SUBG.02
sexo,5109,None,None,F,M
cod_concepto_salarial,5109,30.216284987277355,16.875215739153187,1,7
cod_area,5109,58.252104129966725,21.211345098806586,1,7


### **ATRIBUTOS / LABEL A EMPLEAR EN EL MODELO DE CLASIFICACIÓN**

Se utilizará los siguientes atributos en el modelo de clasificación:

## <font color='green'>**FEATURES** </font>: 

<font color='blue'>**edad**</font>       =   <font color='magenta'>(DATO NUMÉRICO)</font>

<font color='blue'>**sexo**</font>       =   <font color='magenta'>(DATO CATEGÓRICO - BINARIO)</font>
     
<font color='blue'>**salario**</font>      =   <font color='magenta'>(DATO NUMÉRICO)</font>
 
<font color='blue'>**provincia**</font>      =  <font color='magenta'>(DATO CATEGÓRICO - ONE HOT ENCODER)</font>
 
<font color='blue'>**cod_concepto_salarial**</font>     =  <font color='magenta'>(DATO CATEGÓRICO - ONE HOT ENCODER)</font>
     
<font color='blue'>**estado_civil**</font>  =   <font color='magenta'>(DATO CATEGÓRICO - ONE HOT ENCODER)</font>




## <font color='red'>**LABEL** </font>: 

<font color='blue'>**vacaciones**</font>  =  <font color='magenta'>(DATO NUMÉRICO)</font>

##### Asignación de Atributos a evaluar en el dataset :

In [23]:
df = df[['edad', 'sexo', 'salario', 'provincia', 'cod_concepto_salarial', 'estado_civil', 'dias_vacaciones']]

In [24]:
df.printSchema()

root
 |-- edad: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- salario: decimal(38,18) (nullable = true)
 |-- provincia: integer (nullable = true)
 |-- cod_concepto_salarial: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- dias_vacaciones: float (nullable = true)



## Transformación de datos del Dataset

### Variables categóricas (DATOS BINARIOS)

Del conjunto de datos se identifican las siguientes variables categoricas a convertir en datos binarios:

**sexo (Atributo)**


La convención a utilizar será:

**<font color='blue'>M</font>** = <font color='blue'>1</font>

**<font color='blue'>F</font>** = <font color='blue'>0</font>

* **Atributo: sexo**

In [25]:
df = df.withColumn('sexo', F.when(F.col('sexo') == "M", 1).otherwise(F.col('sexo')))

df = df.withColumn('sexo', F.when(F.col('sexo') == "F", 0).otherwise(F.col('sexo')))

- Verificación de la conversión a variable categórica (dato binario) del atributo 'sexo':

In [26]:
df.groupBy('sexo').count().show()

+----+-----+
|sexo|count|
+----+-----+
|   0| 2330|
|   1| 2779|
+----+-----+



Existe una población casi del 50% de datos de cada sexo

- **Cambio de tipo de dato a la columna sexo:**

Para poder aplicar el vector Ensamblador al atributo 'sexo', se requiere que esta columna que tipo String, sea de tipo numérica. Por este motivo, se efectúa el cambio de tipo de columna a integer.

In [27]:
df = df.withColumn('sexo',F.col('sexo').cast('integer'))

In [28]:
df.printSchema()

root
 |-- edad: integer (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- salario: decimal(38,18) (nullable = true)
 |-- provincia: integer (nullable = true)
 |-- cod_concepto_salarial: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- dias_vacaciones: float (nullable = true)



## Feature Engineering

### Cambio de nombre de columna: dias_vacaciones por <font color='red'>**label** </font>

In [31]:
df = df.withColumnRenamed('dias_vacaciones', 'label')

- Visualización del esquema del DataFrame por el cambio en nombre columna:

In [32]:
df.printSchema()

root
 |-- edad: integer (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- salario: decimal(38,18) (nullable = true)
 |-- provincia: integer (nullable = true)
 |-- cod_concepto_salarial: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- label: float (nullable = true)



### Variables categóricas (tipo One Hot Encoder)

Del conjunto de datos se identifican las siguientes variables categoricas a convertir en datos numéricos (One Hot Encoder):

**provincia (Atributo):** 7 valores posibles

**cod_concepto_salarial (Atributo):** 9 valores posibles

**estado_civil (Atributo):** 5 valores posibles

- Importación de Librerías requeridas para conversión de variable categórica a varible numérica tipo One Hot Encoder

In [33]:
#converting categorical data to numerical form

#import required libraries
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

#### One Hot Encoder para atributo *provincia*

- Se define el vector de transformación para el atributo 

In [34]:
#definición del vector encoder para el atributo
provincia_indexer = StringIndexer(inputCol="provincia", outputCol="provincia_index").fit(df)
df = provincia_indexer.transform(df)
provincia_encoder = OneHotEncoder(inputCol="provincia_index", outputCol="provincia_vec")
df = provincia_encoder.transform(df)

- Visualización de vector orginal y vectores transformadores:

In [35]:
df.select(['provincia','provincia_index','provincia_vec']).show(10,False)

+---------+---------------+-------------+
|provincia|provincia_index|provincia_vec|
+---------+---------------+-------------+
|1        |0.0            |(6,[0],[1.0])|
|6        |4.0            |(6,[4],[1.0])|
|1        |0.0            |(6,[0],[1.0])|
|1        |0.0            |(6,[0],[1.0])|
|1        |0.0            |(6,[0],[1.0])|
|1        |0.0            |(6,[0],[1.0])|
|1        |0.0            |(6,[0],[1.0])|
|3        |2.0            |(6,[2],[1.0])|
|1        |0.0            |(6,[0],[1.0])|
|1        |0.0            |(6,[0],[1.0])|
+---------+---------------+-------------+
only showing top 10 rows



#### One Hot Encoder para atributo *cod_concepto_salarial*

In [36]:
#definición del vector encoder para el atributo
ccsalarial_indexer = StringIndexer(inputCol="cod_concepto_salarial", outputCol="ccsalarial_index").fit(df)
df = ccsalarial_indexer.transform(df)
ccsalarial_encoder = OneHotEncoder(inputCol="ccsalarial_index", outputCol="ccsalarial_vec")
df = ccsalarial_encoder.transform(df)

In [37]:
df.select(['cod_concepto_salarial','ccsalarial_index','ccsalarial_vec']).show(10,False)

+---------------------+----------------+--------------+
|cod_concepto_salarial|ccsalarial_index|ccsalarial_vec|
+---------------------+----------------+--------------+
|41                   |0.0             |(8,[0],[1.0]) |
|41                   |0.0             |(8,[0],[1.0]) |
|41                   |0.0             |(8,[0],[1.0]) |
|41                   |0.0             |(8,[0],[1.0]) |
|4                    |5.0             |(8,[5],[1.0]) |
|3                    |3.0             |(8,[3],[1.0]) |
|41                   |0.0             |(8,[0],[1.0]) |
|41                   |0.0             |(8,[0],[1.0]) |
|41                   |0.0             |(8,[0],[1.0]) |
|41                   |0.0             |(8,[0],[1.0]) |
+---------------------+----------------+--------------+
only showing top 10 rows



#### One Hot Encoder para atributo *estado_civil*

In [38]:
#definición del vector encoder para el atributo
estado_civil_indexer = StringIndexer(inputCol="estado_civil", outputCol="estado_civil_index").fit(df)
df = estado_civil_indexer.transform(df)
estado_civil_encoder = OneHotEncoder(inputCol="estado_civil_index", outputCol="estado_civil_vec")
df = estado_civil_encoder.transform(df)

In [39]:
df.select(['estado_civil','estado_civil_index','estado_civil_vec']).show(10,False)

+------------+------------------+----------------+
|estado_civil|estado_civil_index|estado_civil_vec|
+------------+------------------+----------------+
|S           |1.0               |(4,[1],[1.0])   |
|C           |0.0               |(4,[0],[1.0])   |
|C           |0.0               |(4,[0],[1.0])   |
|S           |1.0               |(4,[1],[1.0])   |
|C           |0.0               |(4,[0],[1.0])   |
|S           |1.0               |(4,[1],[1.0])   |
|C           |0.0               |(4,[0],[1.0])   |
|C           |0.0               |(4,[0],[1.0])   |
|S           |1.0               |(4,[1],[1.0])   |
|S           |1.0               |(4,[1],[1.0])   |
+------------+------------------+----------------+
only showing top 10 rows



* Validación de columnas en el DF después de la aplicación del OHE:

In [40]:
df.columns

['edad',
 'sexo',
 'salario',
 'provincia',
 'cod_concepto_salarial',
 'estado_civil',
 'label',
 'provincia_index',
 'provincia_vec',
 'ccsalarial_index',
 'ccsalarial_vec',
 'estado_civil_index',
 'estado_civil_vec']

In [41]:
df.printSchema()

root
 |-- edad: integer (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- salario: decimal(38,18) (nullable = true)
 |-- provincia: integer (nullable = true)
 |-- cod_concepto_salarial: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- label: float (nullable = true)
 |-- provincia_index: double (nullable = false)
 |-- provincia_vec: vector (nullable = true)
 |-- ccsalarial_index: double (nullable = false)
 |-- ccsalarial_vec: vector (nullable = true)
 |-- estado_civil_index: double (nullable = false)
 |-- estado_civil_vec: vector (nullable = true)



* Uso del "VectorAssembler" para crear un único vector de features y clase, para ser usado en el modelo de entrenamiento:

* Construcción del vector ensamblador:

In [42]:
# Vector Ensamblador

# VectorAssembler" para crear un único vector de features y clase, para ser usado por nuestro modelo de entrenamiento
from pyspark.ml.feature import VectorAssembler

# Construcción del vector:
df_assembler = VectorAssembler(inputCols=['edad',
 'sexo',
 'salario',
 'provincia_vec',
 'ccsalarial_vec',
 'estado_civil_vec'], outputCol="features")
df = df_assembler.transform(df)

In [64]:
# visulizacion de vector ensamblado compuesto por features y label

df.select(['features','label']).show(10,False)

+-----------------------------------------------------+-----+
|features                                             |label|
+-----------------------------------------------------+-----+
|(21,[0,1,2,3,9,18],[48.0,1.0,1222186.43,1.0,1.0,1.0])|31.0 |
|(21,[0,1,2,7,9,17],[41.0,1.0,464802.17,1.0,1.0,1.0]) |13.0 |
|(21,[0,1,2,3,9,17],[30.0,1.0,361921.41,1.0,1.0,1.0]) |5.0  |
|(21,[0,2,3,9,18],[37.0,464802.17,1.0,1.0,1.0])       |0.0  |
|(21,[0,1,2,3,14,17],[61.0,1.0,176562.82,1.0,1.0,1.0])|36.0 |
|(21,[0,2,3,12,18],[44.0,190473.26,1.0,1.0,1.0])      |5.0  |
|(21,[0,1,2,3,9,17],[43.0,1.0,464802.17,1.0,1.0,1.0]) |15.0 |
|(21,[0,2,5,9,17],[50.0,464802.17,1.0,1.0,1.0])       |30.0 |
|(21,[0,1,2,3,9,18],[25.0,1.0,464802.17,1.0,1.0,1.0]) |13.0 |
|(21,[0,2,3,9,18],[34.0,574154.48,1.0,1.0,1.0])       |4.5  |
+-----------------------------------------------------+-----+
only showing top 10 rows



## Training Set - Testing Set

* Partición del set de datos en:

75% training 

25% testing:

In [45]:
# Particion del data set

#select data for building model
model_df=df.select(['features','label'])

#split the data 
train,test = model_df.randomSplit([0.75,0.25])

print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 3847
Size of test Dataset : 1262


## 1) Evaluación predictivo con Modelo Linear Regression 

* Importación de librerías correspondientes a **LinearRegression**, así como las de OneHotEncoder, StringIndexer, VectorAssembler:

In [46]:
# Importacion de libs y operaciones

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

from pyspark.ml.regression import LinearRegression

* Creación del Regresor Lineal: 

In [47]:
lr = LinearRegression()

* Entrenamiento del modelo de regresión lineal:

In [48]:
# Fit the model, le llamamos lr_model
lr_model = lr.fit(train)

* Creación del dataframe de prediciones (*predictions_df*) a partir del modelo de entrenamiento y el conjunto de datos test: 

In [49]:
predictions_df = lr_model.transform(test)

* Visualización del contenido de *predictions_df*:

In [50]:
# visulizacion de predictions_df

predictions_df.show(10)

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|(21,[0,1,2,3,9,17...| 15.0|6.3993041972383935|
|(21,[0,1,2,3,9,17...|  9.0| 6.742107711622191|
|(21,[0,1,2,3,9,17...|  0.0| 6.626163666927315|
|(21,[0,1,2,3,9,17...| 12.0| 6.913509468814089|
|(21,[0,1,2,3,9,17...|  4.5| 7.206288443627118|
|(21,[0,1,2,3,9,17...|  0.0| 6.968967181311115|
|(21,[0,1,2,3,9,17...|  5.0| 7.256312983197889|
|(21,[0,1,2,3,9,17...|  3.0| 7.427714740389787|
|(21,[0,1,2,3,9,17...|  1.0| 8.292756335742402|
|(21,[0,1,2,3,9,17...|  4.5| 8.464158092934301|
+--------------------+-----+------------------+
only showing top 10 rows



* Evaluación del modelo de Regresión Lineal, con los datos de TEST:

In [51]:
# evaluacion del modelo, le llamaremos model_predictions

model_predictions = lr_model.evaluate(test)

* Cálculo el valor de R2:

In [52]:
# valor de R2

lr_model.evaluate(test).r2

0.23100427146959668

* Cálculo el valor del RootMeanSquaredError:

In [53]:
# valor del RootMeanSquaredError (RMSE)

lr_model.evaluate(test).rootMeanSquaredError

11.248871394332603

## 2) Evaluación predictivo con Modelo RandomForestRegressor

* Importación de librerías correspondientes a *RandomForestRegressor*

In [54]:
# import lib

from pyspark.ml.regression import RandomForestRegressor

* Creación del Regresor RF:

In [55]:
# Regresor 

rf = RandomForestRegressor()

* Entrenamiento del modelo:

In [56]:
# Train model, le llamaremos rf_model

rf_model = rf.fit(train)

* Despliegue de las *featureImportances*:

In [57]:
# importances 

rf_model.featureImportances

SparseVector(21, {0: 0.3236, 1: 0.0097, 2: 0.1362, 3: 0.0062, 4: 0.006, 5: 0.0048, 6: 0.0024, 7: 0.0024, 8: 0.0028, 9: 0.401, 10: 0.0283, 11: 0.019, 12: 0.0171, 13: 0.016, 14: 0.0022, 15: 0.0076, 16: 0.0009, 17: 0.0024, 18: 0.0075, 19: 0.0015, 20: 0.0022})

* Despliegue del número de árboles (Num of Trees)

In [58]:
# Numero de Trees

rf_model.getNumTrees

20

* Evaluación del modelo con los datos de entrenamiento, se le llamará **model_predictions**:

In [59]:
# model_predictions

model_predictions = rf_model.transform(test)

* Despliegue de los valores del *model_predictions*

In [60]:
model_predictions.show(10)

+--------------------+-----+-----------------+
|            features|label|       prediction|
+--------------------+-----+-----------------+
|(21,[0,1,2,3,9,17...| 15.0|7.863802366098879|
|(21,[0,1,2,3,9,17...|  9.0|7.930022604194117|
|(21,[0,1,2,3,9,17...|  0.0|5.161891227360012|
|(21,[0,1,2,3,9,17...| 12.0| 8.05787174780413|
|(21,[0,1,2,3,9,17...|  4.5| 8.05787174780413|
|(21,[0,1,2,3,9,17...|  0.0| 6.50443846567549|
|(21,[0,1,2,3,9,17...|  5.0|8.441848458286787|
|(21,[0,1,2,3,9,17...|  3.0|8.536423142312064|
|(21,[0,1,2,3,9,17...|  1.0|8.627394351338422|
|(21,[0,1,2,3,9,17...|  4.5|8.627394351338422|
+--------------------+-----+-----------------+
only showing top 10 rows



* Se importa el **RegressionEvaluator**

In [62]:
# import Evaluator

from pyspark.ml.evaluation import RegressionEvaluator

* Usando *RegressionEvaluator*, se calcula e imprime el valor de las métricas **R2** y **RMSE**:

In [63]:
# R2 value of the model on test data 
rf_evaluator = RegressionEvaluator(metricName='r2')
rf_r2 = rf_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is: {rf_r2}')
print(" ")


# RMSE value of the model on test data 
rf_evaluator = RegressionEvaluator(metricName='rmse')
rf_rmse = rf_evaluator.evaluate(model_predictions)
print(f'The RMSE of DecisionTreeRegressor is: {rf_rmse}')

The r-square value of DecisionTreeRegressor is: 0.2655984698202041
 
The RMSE of DecisionTreeRegressor is: 10.99293789524329


**Resumen de Precisiones Obtenidas:**

Clasificador |   R2 value   |        RMSE  
------------ | ------------- | ------------- 
Linear Regression | 23,10 % | 11,25 % 
Random Forest Regressor | 26,56 % | 10,99 % 
              |         |        

Como se aprecia en los resultados obtenidos, los porcentajes de acierto logrados con los dos métodos empleados son de un máximo de 26,56 %, lo que indica una predicción pobre.

La interpretación obtenida sobre estos resultados que utiliza una base de datos real de un banco , es que se tiene atributos que **NO SON importantes** en el motivo que tiene un empleado para tomar vacaciones.

Hubiese sido interesante incluir otros atributos de datos que puedan mejorar estos registros como:

- Cantidad de hijos: el tener hijos ayuda a que el empleado disponga de ciertos días para poder compartir con los hijos (por lo que la edad de esos hijos también influiría en los resultyados que se pudiera obtener).

- Codigo de planilla: a pesar de que viene dentro de los datos iniciales de la BD, se descartó por el hecho de que solamente viene incluido como si todas las personas pertenecieran a un solo código de planilla (fijo).

- Jornada de trabajo: al igual que el código de planilla, solo viene indicado como un solo valor. Pero este atributo define en buena parte si una persona toma o no vacaciones (no es lo mismo trabajar un fin de semana, que hacerlo de día ó de forma mixta).